# IMPORTS

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import precision_score, recall_score
# from sklearn.metrics import confusion_matrix
import my_pickle as mp
from my_resample import div_count_pos_neg, undersample, oversample
import my_features as mf

# UNPICKLE

In [2]:
user_df = mp.unpickle_it('user_df')
convo_df = mp.unpickle_it('convo_df')

print("Make sure we're not useing ECT data")
print(pd.to_datetime(convo_df.timestamp.max()*1000000))

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


In [19]:
X = mp.unpickle_it('X')
print("Make sure we're not useing ECT data")
print(pd.to_datetime(X.timestamp.max()*1000000))

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


# GET X AND y

In [13]:
X = mf.feature_time(convo_df, user_df)
print(X.columns)

10937
10937
Index(['conv_id', 'response', 'first_uid', 'second_uid', 'first_mid',
       'second_mid', 'timestamp', 'age_dif', 'rent_overlap',
       'inverse_distance', 'roommate_num_sim', 'same_gender', 'same_relate',
       'same_clean', 'same_night', 'same_student', 'same_smoking', 'same_term',
       'sender_attractiveness', 'receiver_selectivity'],
      dtype='object')


In [20]:
y = X.response.map({True:1,False:0})
X = X.drop(['conv_id', 'response', 'first_uid', 'second_uid', 'first_mid',
       'second_mid', 'timestamp'], axis=1)
print(len(X))
print(len(y))#.sum()

10937
10937


In [21]:
X.describe()

,age_dif,rent_overlap,roommate_num_sim,same_gender,same_relate,same_clean,same_night,same_student,same_smoking,same_term,sender_attractiveness,receiver_selectivity
count,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000,10937.000000
mean,5.391606,215.324129,1.138978,0.584712,0.577032,0.555911,0.553900,0.439152,0.568895,0.235988,0.107128,0.064989
std,6.939003,182.554362,0.854907,0.492794,0.494053,0.496887,0.497109,0.496306,0.495253,0.424634,0.158821,0.332871
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.000000
25%,2.000000,175.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,175.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.059259,0.000000
75%,7.000000,200.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.155556,0.000000
max,98.000000,3700.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986301


# OVERSAMPLE

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

# X_ove, y_ove = oversample(X.as_matrix(), y.as_matrix(), .5)

# X_train, X_test, y_train, y_test = train_test_split(X_ove, y_ove, random_state=17)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

messages sent: 14708
responses:     7354

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]...
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]...
Model recall: 0.36303630363036304
Model precision: 0.4888888888888889
Score of model: 0.8873857404021938
Score to beat: 0.8892138939670933


In [ ]:
messages sent: 14708
responses:     7354

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]...
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]...
Model recall: 0.31683168316831684
Model precision: 0.5052631578947369
Score of model: 0.889945155393053
Score to beat: 0.8892138939670933

# BASELINE RANDOM FOREST

In [ ]:
print("Messages with responses:    {}".format((convo_df.response == True).sum()))
print("Messages without responses: {}".format((convo_df.response == False).sum()))
print("Total messages:             {}".format(len(convo_df)))

In [ ]:
# X_unsamp, y_unsamp = X.as_matrix(), y.as_matrix()

# X_train, X_test, y_train, y_test = train_test_split(X_unsamp, y_unsamp, random_state=17)

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

# UNDERSAMPLE

In [18]:
# X_und, y_und = undersample(X, y, .5)

# X_train, X_test, y_train, y_test = train_test_split(X_und, y_und, random_state=17)

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = oversample(X_train, y_train, .5)

print("messages sent: {}".format(len(y_train)))
print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}...".format(model.predict(X_test)[:20]))
print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

messages sent: 14708
responses:     7354

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]...
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]...
Model recall: 0.33663366336633666
Model precision: 0.53125
Score of model: 0.8936014625228519
Score to beat: 0.8892138939670933


# OTHER MODELS KEPT HERE

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

In [ ]:
model = GradientBoostingClassifier()

model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))